In [61]:
import cv2
import numpy as np


In [62]:
# 讀取圖像和模板
img_rgb = cv2.imread('mario.jpg')
# img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
template = cv2.imread('marioCoin.jpg', 1)
h, w, _ = template.shape[::]


從v0的版本確定TM_CCOEFF_NORMED的效果適合這張照片的檢測

Determine from version v0 that the effect of TM_CCOEFF_NORMED is suitable for detecting this photo

In [63]:
# 進行模板匹配
# res = cv2.matchTemplate(img_rgb, template, cv2.TM_SQDIFF) #|平方差匹配     |TM_SQDIFF        |1 |minVal|
# res = cv2.matchTemplate(img_rgb, template, cv2.TM_SQDIFF_NORMED) #|標準平方差匹配 |TM_SQDIFF_NORMED |2 |minVal|
# res = cv2.matchTemplate(img_rgb, template, cv2.TM_CCORR) #|相關匹配       |TM_CCORR         |3 |maxVal|
# res = cv2.matchTemplate(img_rgb, template, cv2.TM_CCORR_NORMED) #|標準相關匹配   |TM_CCORR_NORMED  |4 |maxVal|
# res = cv2.matchTemplate(img_rgb, template, cv2.TM_CCOEFF) #|相關匹配       |TM_CCOEFF        |5 |maxVal|
res = cv2.matchTemplate(img_rgb, template, cv2.TM_CCOEFF_NORMED) #|標準相關匹配   |TM_CCOEFF_NORMED |6 |-1 ~ 1 (maxVal)|



In [64]:
# 設定閾值
threshold = .95
loc = np.where(res >= threshold)



使用zeros_like一樣會有相同的框選問題 OpenCV_matchTemplate_v0 有測試

Using zeros_like will have the same bounding box issue, OpenCV_matchTemplate_v0 has been tested

In [65]:
# 創建一個用於標記是否已經被框選的二維陣列
# marked = np.zeros_like(res, dtype=bool)

用pt_x-pt[0]下面的方式還是會有重複框選的問題

714 : 207; 
785 : 207; 
713 : 208; 
784 : 208; 
714 : 209; 
784 : 209; 
820 : 279 

In [66]:
# pt_x=0
# pt_y=0
# count = 0# 初始化框選的數量
# for idx, pt in enumerate(zip(*loc[::-1])):
#     # print(f'match rectangle top_left : {pt}')
#     if (abs(pt_x-pt[0]) + abs(pt_y-pt[1])) <10:
#         continue

#     print(f"{pt[0]} : {pt[1]}")
#     cv2.rectangle(img_rgb, pt, (pt[0]+w, pt[1]+h), (0, 255, 255), 2)
#     pt_x=pt[0]
#     pt_y=pt[1]
#     count += 1


改成先存入[], 下面是驗證測試

Change to store in [] first, the following is the verification test

In [67]:
test_a=5
test_b=6
test_list=[[2,3],[5,6],[8,9]]
print(test_list[0][1])
test_list.append([6,7])
print(test_list)
print(len(test_list))
for i in test_list:
    
    if abs(i[0]-test_a) == 0 and abs(i[1]-test_b) == 0:
        continue
    print(i[0],i[1])

3
[[2, 3], [5, 6], [8, 9], [6, 7]]
4
2 3
8 9
6 7


In [68]:
count = 0 # 初始化框選的數量
pt_temp=[]
for idx, pt in enumerate(zip(*loc[::-1])):
    # print(f'match rectangle top_left : {pt}')
    check = 0
    for i in pt_temp:
        if abs(i[0] -pt[0]) < 10 and abs(i[1]-pt[1]) < 10:
            print(f" {i[0]},{pt[0]} and {i[1]},{pt[1]}")
            check = 1
            break 
        
    if check == 1:
        continue
    # print(f"{pt[0]} : {pt[1]}")
    cv2.rectangle(img_rgb, pt, (pt[0]+w, pt[1]+h), (0, 255, 255), 2)
    pt_temp.append([pt[0],pt[1]])
    print(pt_temp)
    
    count += 1

[[713, 203]]
 713,714 and 203,203
 713,715 and 203,203
 713,716 and 203,203
[[713, 203], [784, 203]]
 784,785 and 203,203
 784,786 and 203,203
 713,712 and 203,204
 713,713 and 203,204
 713,714 and 203,204
 713,715 and 203,204
 713,716 and 203,204
 784,783 and 203,204
 784,784 and 203,204
 784,785 and 203,204
 784,786 and 203,204
 784,787 and 203,204
 713,711 and 203,205
 713,712 and 203,205
 713,713 and 203,205
 713,714 and 203,205
 713,715 and 203,205
 713,716 and 203,205
 713,717 and 203,205
 784,782 and 203,205
 784,783 and 203,205
 784,784 and 203,205
 784,785 and 203,205
 784,786 and 203,205
 784,787 and 203,205
 713,711 and 203,206
 713,712 and 203,206
 713,713 and 203,206
 713,714 and 203,206
 713,715 and 203,206
 713,716 and 203,206
 713,717 and 203,206
 784,782 and 203,206
 784,783 and 203,206
 784,784 and 203,206
 784,785 and 203,206
 784,786 and 203,206
 784,787 and 203,206
 784,788 and 203,206
 713,711 and 203,207
 713,712 and 203,207
 713,713 and 203,207
 713,714 and 203,

設定putText 相關參數

Set the parameters for putText

In [69]:
fontScale = 1 #  color = (0, 255, 0) #  thickness = 2 #  lineType = cv2.LINE_AA # 


org_x = 10 #文字左下角的坐標 Coordinate of the bottom left corner of the text
org_y = 50#文字左下角的坐標 Coordinate of the bottom left corner of the text
fontFace = cv2.FONT_HERSHEY_SIMPLEX #文字的字體 Font of the text
fontScale = 1 #文字的大小 Size of the text
color = (0, 255, 0) #文字的顏色 Color of the text
thickness = 2 #文字的粗細 Thickness of the text
lineType = cv2.LINE_AA #文字的線條類型 Line type of the text

cv2.putText(img_rgb, f'Get the number of matches: {count}', (org_x, org_y),fontFace, fontScale, color, thickness, lineType)


array([[[156,  57,   7],
        [156,  57,   7],
        [160,  57,   8],
        ...,
        [  9, 169,  15],
        [  7, 166,  16],
        [  5, 164,  14]],

       [[156,  57,   7],
        [156,  57,   7],
        [160,  57,   8],
        ...,
        [  9, 169,  15],
        [  7, 166,  16],
        [  5, 164,  14]],

       [[156,  57,   7],
        [156,  57,   7],
        [160,  57,   8],
        ...,
        [  9, 169,  15],
        [  7, 166,  16],
        [  5, 164,  14]],

       ...,

       [[ 68,  44,  26],
        [ 68,  44,  26],
        [ 68,  44,  26],
        ...,
        [ 64,  43,  21],
        [ 63,  42,  21],
        [ 63,  42,  21]],

       [[ 68,  44,  26],
        [ 68,  44,  26],
        [ 68,  44,  26],
        ...,
        [ 64,  43,  21],
        [ 63,  42,  21],
        [ 63,  42,  21]],

       [[ 69,  45,  27],
        [ 69,  45,  27],
        [ 69,  45,  27],
        ...,
        [ 65,  44,  22],
        [ 64,  43,  22],
        [ 64,  43,  22]]

In [70]:
print('Number of boxes:', count)
# 顯示結果
cv2.imshow('Detected', img_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of boxes: 20
